# AFRped data

In [1]:
versioninfo()

Julia Version 1.0.3
Commit 099e826241 (2018-12-18 01:34 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin14.5.0)
  CPU: Intel(R) Core(TM) i9-9880H CPU @ 2.30GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-6.0.0 (ORCJIT, skylake)


In [2]:
using Revise
using DelimitedFiles
using LinearAlgebra
using BenchmarkTools
using MendelImpute
using Random
using Profile

┌ Info: Recompiling stale cache file /Users/biona001/.julia/compiled/v1.0/MendelImpute/DVXpm.ji for MendelImpute [e47305d1-6a61-5370-bc5d-77554d143183]
└ @ Base loading.jl:1190


## Retrieve genotype data from pedigree file

In [3]:
@time rawdata = readdlm("AFRped_geno.txt", ',', Float32)

  7.975786 seconds (176.39 M allocations: 4.777 GiB, 7.91% gc time)


1325×36499 Array{Float32,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  1.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  1.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0 

In [4]:
people = 664
X = copy(Transpose(rawdata[1:people, 1:(end - 1)]))

36498×664 Array{Float32,2}:
 0.0  0.0  1.0  0.0  1.0  1.0  0.0  1.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  

In [5]:
function create_hap(x)
    n, p = size(x)
    h = one(eltype(x))
    for j in 1:p, i in 1:n
        if x[i, j] != 0
            x[i, j] -= h
        end
    end
    return copy(Transpose(x))
end
H = create_hap(rawdata[(people + 1):end, 1:(end - 1)])     

36498×661 Array{Float32,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  

## Phase + impute by moving window (using flanking windows)

In [6]:
Random.seed!(123)
missingprop = 0.1
p, n = size(X)
X2 = Matrix{Union{Missing, eltype(X)}}(X)
Xm = ifelse.(rand(eltype(X), p, n) .< missingprop, missing, X2)
Xm_original = copy(Xm)

36498×664 Array{Union{Missing, Float32},2}:
 0.0        missing  1.0       0.0       …  0.0       0.0       0.0  0.0     
 0.0       0.0       0.0       0.0          0.0       0.0       0.0   missing
 0.0       0.0       0.0       0.0           missing  0.0       0.0  0.0     
 0.0       0.0       0.0       0.0          0.0       0.0       0.0  0.0     
  missing  0.0       0.0       0.0          0.0       0.0       0.0  0.0     
 0.0       0.0       0.0       0.0       …  0.0       0.0       0.0  0.0     
 0.0       0.0       0.0       0.0          0.0       0.0       0.0   missing
 0.0       0.0       0.0       0.0          0.0       0.0       0.0  0.0     
 0.0       0.0       0.0       0.0          0.0       0.0       0.0  0.0     
 0.0       0.0       0.0       0.0          0.0       0.0       0.0  0.0     
 0.0       0.0       0.0       0.0       …  0.0       0.0       0.0  0.0     
 0.0       0.0       0.0       0.0          0.0       0.0       0.0   missing
 0.0       0.0      

In [ ]:
@time ph = phase(Xm, H, 400);

Imputing SNPs 1:400
Imputing SNPs 401:800
window length = 1200
happair = (205, 205)
bkpts = (-1, -1)
Imputing SNPs 801:1200
window length = 1200
happair = (205, 205)
bkpts = (-1, -1)
Imputing SNPs 1201:1600
window length = 1200
happair = (205, 205)
bkpts = (-1, -1)
Imputing SNPs 1601:2000
window length = 1200
happair = (205, 205)
bkpts = (-1, -1)
Imputing SNPs 2001:2400
window length = 1200
happair = (205, 205)
bkpts = (-1, -1)
Imputing SNPs 2401:2800
window length = 1200
happair = (205, 205)
bkpts = (-1, -1)
Imputing SNPs 2801:3200
window length = 1200


In [30]:
ph[1]

MendelImpute.HaplotypeMosaicPair(MendelImpute.HaplotypeMosaic(36498, [1, 8401, 9201, 9601, 10801, 11601, 12001, 12401, 14801, 15201, 18401, 18801, 19201, 20801, 21201, 23201, 24001, 24401, 25601, 26001], [205, 24, 9, 128, 205, 2, 205, 194, 74, 194, 160, 286, 194, 136, 205, 49, 19, 137, 14, 205]), MendelImpute.HaplotypeMosaic(36498, [1, 3951, 4001, 5171, 4867, 5201, 6835, 6801, 8754, 8401  …  20801, 21201, 23201, 24001, 24401, 27601, 28001, 28401, 28801, 35601], [205, 264, 83, 379, 306, 236, 490, 205, 403, 51  …  190, 236, 235, 561, 236, 24, 376, 54, 236, 74]))

In [9]:
impute!(Xm, H, ph)

# After imputation, the genotype matrix disagree with original one even on observed entries....?

In [14]:
diff = 0
for j in 1:n, i in 1:p
    if !ismissing(Xm_original[i, j]) && (X[i, j] != Xm[i, j])
        diff += 1
    end
end
diff

54272

In [25]:
missing_idx    = ismissing.(Xm_original)
total_missing  = sum(missing_idx)
missing_true   = round.(Int, X[missing_idx])  #true values of missing entries
missing_impute = round.(Int, Xm[missing_idx]) #imputed values of missing entries
error = sum(missing_true .!= missing_impute) / total_missing

0.0039437771631928974

## Profiling

In [ ]:
Profile.clear()
@profile phase(Xm, H, 400)
Profile.print(format=:flat)

Imputing SNPs 1:400
Imputing SNPs 401:800
happair = (205, 205)
bkpts = (-1, -1)
Imputing SNPs 801:1200
happair = (205, 205)
bkpts = (-1, -1)
Imputing SNPs 1201:1600
happair = (205, 205)
bkpts = (-1, -1)
Imputing SNPs 1601:2000
happair = (205, 205)
bkpts = (-1, -1)
Imputing SNPs 2001:2400
happair = (205, 205)
bkpts = (-1, -1)
Imputing SNPs 2401:2800
happair = (205, 205)
bkpts = (-1, -1)
Imputing SNPs 2801:3200
happair = (205, 205)
bkpts = (-1, -1)
Imputing SNPs 3201:3600
happair = (205, 205)
bkpts = (-1, -1)
Imputing SNPs 3601:4000
happair = (205, 205)
bkpts = (-1, -1)
Imputing SNPs 4001:4400
happair = (205, 264)
bkpts = (-1, 350)
Imputing SNPs 4401:4800
happair = (205, 83)
bkpts = (-1, 0)
Imputing SNPs 4801:5200
happair = (205, 120)
bkpts = (-1, 770)
Imputing SNPs 5201:5600
happair = (205, 338)
bkpts = (-1, 153)
Imputing SNPs 5601:6000
happair = (205, 236)
bkpts = (-1, 0)
Imputing SNPs 6001:6400
happair = (205, 236)
bkpts = (-1, -1)
Imputing SNPs 6401:6800
happair = (205, 515)
bkpts = 

## Effect of window size on accuracy and efficiency

In [15]:
width_range = 100:100:1000
timing = zeros(length(width_range))
errate = zeros(length(width_range))
for i in eachindex(width_range)
    tic()
    copy!(Xm.values, rawdata[1:people, 1:(end - 1)]')
    ph = phase(Xm, H, width_range[i], false)
    impute!(Xm.values, H, ph)
    timing[i] = toq()
    missing_impute = round.(Int, Xm.values[Xm.isnull])
    errate[i] = countnz(missing_true .≠ missing_impute) / length(missing_true)
    println("width = $(width_range[i]), time = $(timing[i]), error rate = $(errate[i])")
end

width = 100, time = 44.652082362, error rate = 0.009299016565174983
width = 200, time = 23.251482066, error rate = 0.00504976976200836
width = 300, time = 16.099439054, error rate = 0.0038203215701962953
width = 400, time = 12.630925796, error rate = 0.0036607118921582983
width = 500, time = 11.166686167, error rate = 0.0037988753085477786
width = 600, time = 9.042634645, error rate = 0.004253783512361502
width = 700, time = 8.137103798, error rate = 0.004584550855479004
width = 800, time = 7.262145625, error rate = 0.005146690367535309
width = 900, time = 6.735577601, error rate = 0.005656039081687573
width = 1000, time = 6.250194499, error rate = 0.006445838909705052


In [16]:
using Plots
gr()

plot(width_range, timing; xlabel = "Window width", 
    ylabel = "Run time in seconds", leg = false)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 250 
 
 
 500 
 
 
 750 
 
 
 1000 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 Window width 
 
 
 Run time in seconds

In [17]:
plot(width_range, errate; xlabel = "Window width", 
    ylabel = "Error rate", leg = false)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 250 
 
 
 500 
 
 
 750 
 
 
 1000 
 
 
 0.0050 
 
 
 0.0075 
 
 
 Window width 
 
 
 Error rate